In [2]:
import pandas as pd
import seaborn as sns

In [3]:
df = pd.read_csv('COVID-19_vaccination_rates.csv') 

In [4]:
countries = ["Austria", "Germany", "Portugal","Spain", "Switzerland"] # add more countries
df.loc[(df["location"].isin(countries))].loc[(df["date"]=="2021-10-25")]  # specific date?!
#df.loc[(df["country"].isin(countries))& (df["date"]==2020-10-27)]

,location,iso_code,date,total_vaccinations,people_vaccinated,people_fully_vaccinated,total_boosters,daily_vaccinations_raw,daily_vaccinations,total_vaccinations_per_hundred,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,total_boosters_per_hundred,daily_vaccinations_per_million,daily_people_vaccinated,daily_people_vaccinated_per_hundred
4216,Austria,AUT,2021-10-25,11442514.0,5878560.0,5609062.0,279704.0,15949.0,21939.0,126.53,65.01,62.03,3.09,2426.0,5466.0,0.060
22709,Germany,DEU,2021-10-25,111097937.0,57559637.0,55181203.0,1651090.0,107670.0,130759.0,132.42,68.60,65.77,1.97,1559.0,33209.0,0.040
48803,Portugal,PRT,2021-10-25,16222829.0,9017464.0,8865394.0,NaN,NaN,6164.0,159.55,88.69,87.19,NaN,606.0,1945.0,0.019
56871,Spain,ESP,2021-10-25,71366561.0,37988609.0,37250048.0,483728.0,22084.0,42043.0,152.67,81.27,79.69,1.03,899.0,12721.0,0.027
58392,Switzerland,CHE,2021-10-25,11030897.0,5722003.0,5483891.0,275.0,16793.0,19190.0,126.57,65.65,62.92,0.00,2202.0,5120.0,0.059


In [5]:
df0 = pd.read_csv('gdp.csv')  ## divide by population

In [6]:
df0.loc[(df0["Country Name"].isin(countries))]

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
14,Austria,AUT,GDP (current US$),NY.GDP.MKTP.CD,6.592694e+09,7.311750e+09,7.756110e+09,8.374175e+09,9.169984e+09,9.994071e+09,...,4.311203e+11,4.094252e+11,4.300687e+11,4.419961e+11,3.818176e+11,3.955686e+11,4.162073e+11,4.548726e+11,4.450754e+11,4.309474e+11
37,Switzerland,CHE,GDP (current US$),NY.GDP.MKTP.CD,9.522747e+09,1.071271e+10,1.187998e+10,1.306364e+10,1.448056e+10,1.534674e+10,...,7.220382e+11,6.921097e+11,7.127481e+11,7.343966e+11,7.021496e+11,6.956007e+11,7.044785e+11,7.355393e+11,7.317674e+11,7.522480e+11
55,Germany,DEU,GDP (current US$),NY.GDP.MKTP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,3.744409e+12,3.527345e+12,3.732743e+12,3.883920e+12,3.356236e+12,3.467498e+12,3.681733e+12,3.975347e+12,3.888327e+12,3.846414e+12
70,Spain,ESP,GDP (current US$),NY.GDP.MKTP.CD,1.207213e+10,1.383430e+10,1.613855e+10,1.907491e+10,2.134384e+10,2.475696e+10,...,1.478773e+12,1.324820e+12,1.354757e+12,1.369399e+12,1.195119e+12,1.232076e+12,1.309297e+12,1.420300e+12,1.393046e+12,1.281485e+12
194,Portugal,PRT,GDP (current US$),NY.GDP.MKTP.CD,3.193200e+09,3.417517e+09,3.668222e+09,3.905734e+09,4.235608e+09,4.687464e+09,...,2.447972e+11,2.162366e+11,2.263695e+11,2.295962e+11,1.993139e+11,2.062860e+11,2.208111e+11,2.421948e+11,2.395108e+11,2.312267e+11


In [7]:
df1 = pd.read_csv('unemployment_rate.csv') 

In [8]:
df1.loc[(df1["Country Name"].isin(countries))]

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
14,Austria,AUT,"Unemployment, total (% of total labor force) (...",SL.UEM.TOTL.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,4.56,4.87,5.34,5.62,5.72,6.01,5.50,4.85,4.49,5.77
37,Switzerland,CHE,"Unemployment, total (% of total labor force) (...",SL.UEM.TOTL.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,4.40,4.49,4.75,4.83,4.80,4.92,4.80,4.71,4.39,4.94
55,Germany,DEU,"Unemployment, total (% of total labor force) (...",SL.UEM.TOTL.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,5.82,5.38,5.23,4.98,4.62,4.12,3.75,3.38,3.14,4.31
70,Spain,ESP,"Unemployment, total (% of total labor force) (...",SL.UEM.TOTL.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,21.39,24.79,26.09,24.44,22.06,19.64,17.22,15.26,14.10,15.67
194,Portugal,PRT,"Unemployment, total (% of total labor force) (...",SL.UEM.TOTL.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,12.68,15.53,16.18,13.89,12.44,11.07,8.87,6.99,6.46,7.20


In [9]:
df2 = pd.read_csv('political_regime.csv')   ## NOT HAPPY
df2.loc[(df2["Country"].isin(countries))]

,Case Name,Country,beg yr,end yr,Regime type,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9
25,Austria 45-NA,Austria,1946.0,NaN,democracy,NaN,NaN,NaN,NaN,NaN
364,Portugal 26-74,Portugal,1927.0,1974,personal,NaN,NaN,NaN,NaN,NaN
365,Portugal 74-76,Portugal,1975.0,1976,provisional,NaN,NaN,NaN,NaN,NaN
366,Portugal 76-NA,Portugal,1977.0,NaN,democracy,NaN,NaN,NaN,NaN,NaN
399,Spain 39-76,Spain,1940.0,1976,personal,NaN,NaN,NaN,NaN,NaN
400,Spain 76-77,Spain,1977.0,1977,provisional,NaN,NaN,NaN,NaN,NaN
401,Spain 77-NA,Spain,1978.0,NaN,democracy,NaN,NaN,NaN,NaN,NaN
415,Switzerland 1870-NA,Switzerland,1871.0,NaN,democracy,NaN,NaN,NaN,NaN,NaN


In [22]:
df3 = pd.read_csv('trust_in_government.csv') 
locations = ["AUT", "DEU", "PRT","ESP", "CHE"]
df3.loc[(df3["LOCATION"].isin(locations))]

,LOCATION,INDICATOR,SUBJECT,MEASURE,FREQUENCY,TIME,Value,Flag Codes
14,AUT,TRUSTGOV,TOT,PC,A,2006,49.703779,NaN
15,AUT,TRUSTGOV,TOT,PC,A,2008,25.724219,NaN
16,AUT,TRUSTGOV,TOT,PC,A,2009,48.105214,NaN
17,AUT,TRUSTGOV,TOT,PC,A,2010,49.184108,NaN
18,AUT,TRUSTGOV,TOT,PC,A,2011,40.692012,NaN
...,...,...,...,...,...,...,...,...
363,CHE,TRUSTGOV,TOT,PC,A,2016,79.853670,NaN
364,CHE,TRUSTGOV,TOT,PC,A,2017,81.970676,NaN
365,CHE,TRUSTGOV,TOT,PC,A,2018,84.997947,NaN
366,CHE,TRUSTGOV,TOT,PC,A,2019,80.661271,NaN


In [16]:
df4 = pd.read_csv('unemployment_rate.csv') 
df4.loc[(df4["Country Name"].isin(countries))]

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
14,Austria,AUT,"Unemployment, total (% of total labor force) (...",SL.UEM.TOTL.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,4.56,4.87,5.34,5.62,5.72,6.01,5.50,4.85,4.49,5.77
37,Switzerland,CHE,"Unemployment, total (% of total labor force) (...",SL.UEM.TOTL.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,4.40,4.49,4.75,4.83,4.80,4.92,4.80,4.71,4.39,4.94
55,Germany,DEU,"Unemployment, total (% of total labor force) (...",SL.UEM.TOTL.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,5.82,5.38,5.23,4.98,4.62,4.12,3.75,3.38,3.14,4.31
70,Spain,ESP,"Unemployment, total (% of total labor force) (...",SL.UEM.TOTL.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,21.39,24.79,26.09,24.44,22.06,19.64,17.22,15.26,14.10,15.67
194,Portugal,PRT,"Unemployment, total (% of total labor force) (...",SL.UEM.TOTL.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,12.68,15.53,16.18,13.89,12.44,11.07,8.87,6.99,6.46,7.20


In [19]:
df4 = pd.read_csv('higher_education_rate.csv')  ## NOT HAPPY
df4.loc[(df4["Country Name"].isin(countries))]

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
14,Austria,AUT,"Educational attainment, at least Bachelor's or...",SE.TER.CUAT.BA.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,11.873390,13.081670,13.45572,14.248630,NaN,15.36363,NaN
37,Switzerland,CHE,"Educational attainment, at least Bachelor's or...",SE.TER.CUAT.BA.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
55,Germany,DEU,"Educational attainment, at least Bachelor's or...",SE.TER.CUAT.BA.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,24.182949,24.742050,25.58012,NaN,25.149879,NaN,NaN
70,Spain,ESP,"Educational attainment, at least Bachelor's or...",SE.TER.CUAT.BA.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,20.93539,NaN,21.936340,NaN,23.578011
194,Portugal,PRT,"Educational attainment, at least Bachelor's or...",SE.TER.CUAT.BA.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,14.11663,15.23539,15.87279,17.484989,18.182529,18.99614,19.249451,18.654030,20.72377,22.507700
